In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
import tensorflow as tf

In [ ]:
data = pd.read_csv("/dataset.csv")
data

In [ ]:
# calculate revenue
data['revenue'] = data['price'] * data['sales']

# lookup table
lookup_table = data.groupby('product_variant_id').agg({
    'sales': 'mean',
    'views': 'mean',
    'rating': 'mean',
    'revenue': 'mean'
}).reset_index()

# lookup dict
lookup_dict = lookup_table.set_index('product_variant_id').to_dict(orient='index')

# fetch historical data
def fetch_historical_data(product_id):
    if product_id in lookup_dict:
        return lookup_dict[product_id]
    else:
        return {'sales': 0, 'views': 0, 'rating': 0, 'revenue': 0}

In [ ]:
X = data[['product_variant_id', 'price', 'time_period']].copy()
Y = data[['sales', 'views', 'rating', 'revenue']].copy()

In [ ]:
# add historical data to X
X['historical_sales'] = X['product_variant_id'].map(lambda x: fetch_historical_data(x)['sales'])
X['historical_views'] = X['product_variant_id'].map(lambda x: fetch_historical_data(x)['views'])
X['historical_rating'] = X['product_variant_id'].map(lambda x: fetch_historical_data(x)['rating'])
X['historical_revenue'] = X['product_variant_id'].map(lambda x: fetch_historical_data(x)['revenue'])

X = X.drop(columns=['product_variant_id'])

In [ ]:
X

In [ ]:
Y

In [ ]:
# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
# normalize the data (StandardScaler)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

print("X_train_scaled shape:", X_train_scaled.shape)
print("y_train_scaled shape:", y_train_scaled.shape)

In [ ]:
# build the nn model
input_layer = Input(shape=(X_train_scaled.shape[1],), name='input_features')

# Hidden layers
dense_1 = Dense(256, activation='relu', tf.kernel_regularizer=l2(0.01))(input_layer)
dense_2 = Dense(128, activation='relu', tf.kernel_regularizer=l2(0.01))(dense_1)
bn_1 = BatchNormalization()(dense_2)
dense_3 = Dense(64, activation='relu', tf.kernel_regularizer=l2(0.01))(dense_2)
bn_2 = BatchNormalization()(dense_3)
dense_4 = Dense(16, activation='relu', tf.kernel_regularizer=l2(0.01))(dense_3)
bn_3 = BatchNormalization()(dense_4)

# output layers
output_sales = Dense(1, name='sales_output')(dense_4)
output_views = Dense(1, name='views_output')(dense_4)
output_rating = Dense(1, name='rating_output')(dense_4)
output_revenue = Dense(1, name='revenue_output')(dense_4)

# create the model
model = Model(inputs=input_layer, outputs=[output_sales, output_views, output_rating, output_revenue])

In [ ]:
# compile the model (MSE)
model.compile(optimizer='adam',
              loss={'sales_output': 'mse',
                    'views_output': 'mse',
                    'rating_output': 'mse',
                    'revenue_output': 'mse'})

In [ ]:
# training
history = model.fit(X_train_scaled,
                    [y_train_scaled[:, 0], y_train_scaled[:, 1], y_train_scaled[:, 2], y_train_scaled[:, 3]],
                    epochs=20,
                    batch_size=16,
                    validation_split=0.2)

In [ ]:
import numpy as np

# make predictions
predictions = model.predict(X_test_scaled)

# unscale the preds.
pred_sales = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 0]
pred_views = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 1]
pred_rating = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 2]
pred_revenue = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 3]



print(f"Predicted sales: {np.floor(pred_sales).astype(int)}")
print(f"Predicted views: {np.floor(pred_views).astype(int)}")
print(f"Predicted rating: {np.round(pred_rating, 1)}")
print(f"Predicted revenue: {np.round(pred_revenue).astype(int)}")

In [ ]:
def predict_future_performance(product_variant_id, price, time_period, lookup_dict, model, scaler_X, scaler_y):
    # fetch historical data
    historical_data = fetch_historical_data(product_variant_id)

    # prepare the input (user inputs & historical data)
    input_data = np.array([
        price,                             
        time_period,                       
        historical_data['sales'],          
        historical_data['views'],          
        historical_data['rating'],        
        historical_data['revenue']       
    ]).reshape(1, -1)
    print(input_data)
    # scale the input 
    input_data_scaled = scaler_X.transform(input_data)

    # mkae predictions
    predictions = model.predict(input_data_scaled)

    # unscale the predictions
    pred_sales = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 0]
    pred_views = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 1]
    pred_rating = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 2]
    pred_revenue = scaler_y.inverse_transform(np.hstack([predictions[0], predictions[1], predictions[2], predictions[3]]))[:, 3]

    # return predictions in a dictionary format
    return {
      'Predicted sales': np.floor(pred_sales).astype(int),
      "Predicted views": np.floor(pred_views).astype(int),
      "Predicted rating": np.round(pred_rating, 1),
      "Predicted revenue": np.round(pred_revenue).astype(int)
    }

In [ ]:
# example
user_product_variant_id = 770487  
user_price = 1260000          
user_time_period = 7              


predictions = predict_future_performance(user_product_variant_id, user_price, user_time_period, lookup_dict, model, scaler_X, scaler_y)

print(predictions)